In [161]:
import requests

class CraftingItem:
    # Hard-coded recipe IDs for specific item IDs
    HARDCODED_RECIPES = {
        97339: 13584,  # Core tier 1
        97041: 13542,  # Core tier 2
        97284: 13550,  # Core tier 3
        96628: 13821,  # Core tier 4
        95864: 13658,  # Core tier 5
        96467: 13723,  # Core tier 6
        97020: 13751,  # Core tier 7
        96299: 13780,  # Core tier 8
        96070: 13841,  # Core tier 9
        96613: 13628  # Core tier 10
    }

    def __init__(self, item_id):
        self.item_id = item_id
        self.base_ingredients = {}  # Children items and their quantities
        self.recipe_id = None  # Recipe ID for this item
        self.recipe_data = None  # Full recipe data from the API

    @staticmethod
    def api_querier(url, params=None):
        """Static method to query the Guild Wars 2 API."""
        response = requests.get(url, params=params)

        if response.status_code == 200:
            return response.json()
        else:
            raise Exception(f"API request failed with status code {response.status_code}: {response.text}")

    def get_recipe_id(self):
        """Fetch the recipe ID for this item."""
        # Check if the item ID has a hard-coded recipe ID
        if self.item_id in self.HARDCODED_RECIPES:
            self.recipe_id = self.HARDCODED_RECIPES[self.item_id]
            return self.recipe_id

        # Otherwise, query the API
        url = f"https://api.guildwars2.com/v2/recipes/search?output={self.item_id}"
        response = self.api_querier(url)

        if not response:
            return False
        else:
            self.recipe_id = response[0]  # Store the first recipe ID
            # print(self.recipe_id)
            return self.recipe_id

    def get_recipe_data(self):
        """Fetch and store the full recipe data for this item."""
        if not self.recipe_id:
            self.get_recipe_id()  # Ensure we have a recipe ID

        if self.recipe_id:
            url = f"https://api.guildwars2.com/v2/recipes?ids={self.recipe_id}&v=latest&lang=en"
            self.recipe_data = self.api_querier(url)[0]

            # print(self.recipe_data[0]["ingredients"])

            return self.recipe_data
        else:
            return False

    def fetch_ingredients(self):
        """
        Fetch the ingredients (children) for this item from the API.
        If the item has no recipe, it is considered a base (gatherable) item.
        """
        if not self.recipe_data:
            self.get_recipe_data()  # Ensure we have recipe data

        if self.recipe_data and "ingredients" in self.recipe_data:
            # Add children (base ingredients) to this item
            for ingredient in self.recipe_data["ingredients"]:
                # if CraftingItem(ingredient["type"]) == "Item":
                # print(ingredient)
                # print(ingredient["type"])
                if ingredient["type"] in ['Item', 'Currency']:
                    # print(ingredient["type"])
                    child_item = CraftingItem(ingredient["id"])
                    child_item.fetch_ingredients()  # Recursively fetch children
                    self.base_ingredients[child_item] = ingredient["count"]
        else:
            # This is a base (gatherable) item with no children
            self.base_ingredients = {}

    def get_total_ingredients(self):
        """
        Recursively calculate the total base (gatherable) ingredients required to craft this item.
        """
        total_ingredients = {}

        for child, quantity in self.base_ingredients.items():
            if child.base_ingredients:
                # If the child has its own ingredients, recurse
                child_ingredients = child.get_total_ingredients()
                for item_id, count in child_ingredients.items():
                    if item_id in total_ingredients:
                        total_ingredients[item_id] += count * quantity
                    else:
                        total_ingredients[item_id] = count * quantity
            else:
                # If the child is a base (gatherable) item, add it to the total
                if child.item_id in total_ingredients:
                    total_ingredients[child.item_id] += quantity
                else:
                    total_ingredients[child.item_id] = quantity

        return total_ingredients

    def __repr__(self):
        return f"CraftingItem(item_id={self.item_id}, recipe_id={self.recipe_id}, base_ingredients={self.base_ingredients})"

In [146]:
carrion_helmet = CraftingItem(10675)

if carrion_helmet.recipe_data and "ingredients" in carrion_helmet.recipe_data:
    print("yay")

In [147]:
carrion_helmet.get_recipe_data()

carrion_helmet.recipe_data

{'id': 718,
 'type': 'Helm',
 'output_item_id': 10675,
 'output_item_count': 1,
 'time_to_craft_ms': 1000,
 'disciplines': ['Armorsmith'],
 'min_rating': 300,
 'flags': [],
 'ingredients': [{'type': 'Item', 'id': 19865, 'count': 1},
  {'type': 'Item', 'id': 13155, 'count': 1},
  {'type': 'Item', 'id': 13154, 'count': 1}],
 'chat_link': '[&Cc4CAAA=]'}

In [148]:
carrion_helmet.fetch_ingredients()

carrion_helmet.base_ingredients

{CraftingItem(item_id=19865, recipe_id=90, base_ingredients={CraftingItem(item_id=19747, recipe_id=8, base_ingredients={CraftingItem(item_id=19748, recipe_id=None, base_ingredients={}): 3}): 1, CraftingItem(item_id=24341, recipe_id=None, base_ingredients={}): 3}): 1,
 CraftingItem(item_id=13155, recipe_id=559, base_ingredients={CraftingItem(item_id=19684, recipe_id=18, base_ingredients={CraftingItem(item_id=19700, recipe_id=None, base_ingredients={}): 2}): 1}): 1,
 CraftingItem(item_id=13154, recipe_id=556, base_ingredients={CraftingItem(item_id=19747, recipe_id=8, base_ingredients={CraftingItem(item_id=19748, recipe_id=None, base_ingredients={}): 3}): 2, CraftingItem(item_id=19791, recipe_id=None, base_ingredients={}): 1}): 1}

In [151]:
carrion_helmet = CraftingItem(10675)


# Fetch the ingredients for the sword and its children
carrion_helmet.fetch_ingredients()

# Get the total base (gatherable) ingredients required to craft the sword
total_ingredients = carrion_helmet.get_total_ingredients()
print("Total Base Ingredients:", total_ingredients)

Total Base Ingredients: {19748: 9, 24341: 3, 19700: 2, 19791: 1}


In [160]:
jade_core_6 = CraftingItem(96467)

jade_core_6.recipe_id = 13723

# Fetch the ingredients for the sword and its children
jade_core_6.fetch_ingredients()

# Get the total base (gatherable) ingredients required to craft the sword
total_ingredients = jade_core_6.get_total_ingredients()
print("Total Base Ingredients:", total_ingredients)

Total Base Ingredients: {61: 193, 19697: 70, 19704: 5, 97487: 6, 19699: 150, 19703: 50, 19750: 25, 19698: 50}


In [38]:
test = {'id': 718, 'type': 'Helm', 'output_item_id': 10675, 'output_item_count': 1, 'time_to_craft_ms': 1000, 'disciplines': ['Armorsmith'], 'min_rating': 300, 'flags': [], 'ingredients': [{'type': 'Item', 'id': 19865, 'count': 1}, {'type': 'Item', 'id': 13155, 'count': 1}, {'type': 'Item', 'id': 13154, 'count': 1}], 'chat_link': '[&Cc4CAAA=]'}

In [40]:
test['ingredients']

[{'type': 'Item', 'id': 19865, 'count': 1},
 {'type': 'Item', 'id': 13155, 'count': 1},
 {'type': 'Item', 'id': 13154, 'count': 1}]

In [46]:
for ingredient in test["ingredients"]:
    print(ingredient["type"])
    if ingredient["type"] == 'Item':
        print(1)

Item
1
Item
1
Item
1


In [43]:
test['ingredients'][0]['type']

'Item'

In [10]:
total_ingredients = carrion_helmet.get_total_ingredients()

In [13]:
total_ingredients

{19865: 1, 13155: 1, 13154: 1}